In [2]:
import os
import numpy as np
import pandas as pd
from typing import Tuple
from pyimzml.ImzMLParser import ImzMLParser

In [6]:
# Define folder that contains the dhg dataset
DHG_PATH = os.path.join("..", "DHG")
# Define folder that contains raw data
DHG_RAW_DATA = os.path.join(DHG_PATH, "raw")
# Define folder to save processed data
DHG_PROCESSED_DATA = os.path.join(DHG_PATH, "processed")
# Define file that contains dhg metadata
METADATA_PATH = os.path.join(DHG_PATH, "metadata.csv")
# Define file that contains dhg metadata
METADATA_PATH = os.path.join(DHG_PATH, "metadata.csv")
# Read metadata csv
metadata_df = pd.read_csv(METADATA_PATH)

In [3]:
def read_msi(p: ImzMLParser) -> Tuple[np.ndarray, np.ndarray]:
  """
    Function to read a continuos imzML parser object into a numpy array.

    Args:
        p (ImzMLParser): The imzML parser.
    Returns:
        Tuple[np.ndarray, np.ndarray]: Numpy 3D matrix where y coordinate
            (axis=0), x coordinate (axis=1), intensities values (axis=2)
            and continuos mzs values.

    """
  # Get shape of mzs values
  max_z = p.mzLengths[0]
  # Get shape of y axis
  max_y = p.imzmldict["max count of pixels y"]
  # Get shape of x axis
  max_x = p.imzmldict["max count of pixels x"]
  # Create empty numpy 3D matrix
  msi = np.zeros((max_y, max_x, max_z))
  # Loop over each coordinate and add to 3D matrix
  for i, (x, y, _) in enumerate(p.coordinates):
    # Get mzs and intensities
    mzs, ints = p.getspectrum(i)
    # Add intensities to x,y coordinate
    msi[y - 1, x - 1, :] = ints
  return mzs, msi

In [4]:
# Define dict to store average spectra for each biopsy
mean = {}
# Define common mzs
mzs = np.zeros((24000,))
# Loop over each biopsy
for folder in os.listdir(DHG_PROCESSED_DATA):
  # Get segmentation image
  segment_image = np.load(
      os.path.join(DHG_PROCESSED_DATA, folder, "segmentation.npy")
  )
  # Open parser
  with ImzMLParser(
      os.path.join(DHG_PROCESSED_DATA, folder, "meaningful_signal.imzML")
  ) as reader:
    # Read image and get common mzs
    mzs, data = read_msi(reader)
    # Get mean spectra of image
    mean[folder] = np.clip(data[segment_image, :], a_min=0,
                           a_max=None).mean(axis=0)

# Get for each biopsy mean in the lipids region
mean_lipids = {k: v[((mzs >= 600) & (mzs <= 900))] for k, v in mean.items()}

# Get the lipids mean for all biopsies
global_mean_lipids = np.mean(list(mean_lipids.values()), axis=0)

# Get top peaks in the lipids mean for all biopsies
top_k_peaks = global_mean_lipids.argsort()[-1000:][::-1]

# Get top peaks in the lipids mean for each biopsy
mean_top_k = {k: v[top_k_peaks] for k, v in mean_lipids.items()}

# Define list to store top peaks in the lipids mean for each biopsy per grade and biopsy type
r_high_grade = []
r_low_grade = []
s_high_grade = []
s_low_grade = []

# Sort top peaks in the lipids mean
for k, v in mean_top_k.items():
  if 's' in k:
    if (metadata_df[metadata_df.sample_file_name == k].who_grade <= 2).sum():
      s_low_grade.append(v)
    else:
      s_high_grade.append(v)
  else:
    if (metadata_df[metadata_df.sample_file_name == k].who_grade <= 2).sum():
      r_low_grade.append(v)
    else:
      r_high_grade.append(v)

# Save to csv
pd.DataFrame(
    {
        'mean': np.mean(r_high_grade, axis=0),
        'std': np.std(r_high_grade, axis=0),
        'count': len(r_high_grade)
    }, index=mzs[((mzs >= 600) & (mzs <= 900))][top_k_peaks]
).to_csv("replica_high_grade.csv")
pd.DataFrame(
    {
        'mean': np.mean(r_low_grade, axis=0),
        'std': np.std(r_low_grade, axis=0),
        'count': len(r_low_grade)
    }, index=mzs[((mzs >= 600) & (mzs <= 900))][top_k_peaks]
).to_csv("replica_low_grade.csv")
pd.DataFrame(
    {
        'mean': np.mean(s_high_grade, axis=0),
        'std': np.std(s_high_grade, axis=0),
        'count': len(s_high_grade)
    }, index=mzs[((mzs >= 600) & (mzs <= 900))][top_k_peaks]
).to_csv("section_high_grade.csv")
pd.DataFrame(
    {
        'mean': np.mean(s_low_grade, axis=0),
        'std': np.std(s_low_grade, axis=0),
        'count': len(s_low_grade)
    }, index=mzs[((mzs >= 600) & (mzs <= 900))][top_k_peaks]
).to_csv("section_low_grade.csv")

In [53]:
# Define dict to store average spectra for each biopsy
sum = {}
count = {}
# Define common mzs
mzs = np.zeros((24000,))
# Loop over each biopsy
for folder in os.listdir(DHG_PROCESSED_DATA):
  # Get segmentation image
  segment_image = np.load(
      os.path.join(DHG_PROCESSED_DATA, folder, "segmentation.npy")
  )
  # Open parser
  with ImzMLParser(
      os.path.join(DHG_PROCESSED_DATA, folder, "meaningful_signal.imzML")
  ) as reader:
    # Read image and get common mzs
    mzs, data = read_msi(reader)
    # Get sum spectra of image
    sum[folder] = np.clip(data[segment_image, :], a_min=0,
                           a_max=None).sum(axis=0)
    # Get count of spectras in image
    count[folder] = data[segment_image, :].shape[0]

# Get global mean spectra
mean = np.sum(list(sum.values()), axis=0) / np.sum(list(count.values()), axis=0)

# Get for each biopsy mean in the lipids region
mean_lipids = mean[((mzs >= 600) & (mzs <= 900))]

# Get top peaks in the lipids mean for all biopsies
top_k_peaks = mean_lipids.argsort()[-1000:][::-1]

# Get top 1000 in each spectra
spectras = {}
# Loop over each biopsy
for folder in os.listdir(DHG_PROCESSED_DATA):
  # Get segmentation image
  segment_image = np.load(
      os.path.join(DHG_PROCESSED_DATA, folder, "segmentation.npy")
  )
  # Open parser
  with ImzMLParser(
      os.path.join(DHG_PROCESSED_DATA, folder, "meaningful_signal.imzML")
  ) as reader:
    # Read image and get common mzs
    mzs, data = read_msi(reader)
    # Get mean spectra of image
    spectras[folder] = np.clip(data[segment_image, :], a_min=0,
                           a_max=None)[:, ((mzs >= 600) & (mzs <= 900))][:, top_k_peaks]

# Define list to store top peaks in the lipids mean for each biopsy per grade and biopsy type
r_high_grade = []
r_low_grade = []
s_high_grade = []
s_low_grade = []

# Sort top peaks in the lipids mean
for k, v in spectras.items():
  if 's' in k:
    if (metadata_df[metadata_df.sample_file_name == k].who_grade <= 2).sum():
      s_low_grade.append(v)
    else:
      s_high_grade.append(v)
  else:
    if (metadata_df[metadata_df.sample_file_name == k].who_grade <= 2).sum():
      r_low_grade.append(v)
    else:
      r_high_grade.append(v)

r_high_grade = np.concatenate(r_high_grade)
r_low_grade = np.concatenate(r_low_grade)
s_high_grade = np.concatenate(s_high_grade)
s_low_grade = np.concatenate(s_low_grade)

# Save to csv
pd.DataFrame(
    {
        'mean': np.mean(r_high_grade, axis=0),
        'std': np.std(r_high_grade, axis=0),
        'count': len(r_high_grade)
    }, index=mzs[((mzs >= 600) & (mzs <= 900))][top_k_peaks]
).to_csv("replica_high_grade.csv")
pd.DataFrame(
    {
        'mean': np.mean(r_low_grade, axis=0),
        'std': np.std(r_low_grade, axis=0),
        'count': len(r_low_grade)
    }, index=mzs[((mzs >= 600) & (mzs <= 900))][top_k_peaks]
).to_csv("replica_low_grade.csv")
pd.DataFrame(
    {
        'mean': np.mean(s_high_grade, axis=0),
        'std': np.std(s_high_grade, axis=0),
        'count': len(s_high_grade)
    }, index=mzs[((mzs >= 600) & (mzs <= 900))][top_k_peaks]
).to_csv("section_high_grade.csv")
pd.DataFrame(
    {
        'mean': np.mean(s_low_grade, axis=0),
        'std': np.std(s_low_grade, axis=0),
        'count': len(s_low_grade)
    }, index=mzs[((mzs >= 600) & (mzs <= 900))][top_k_peaks]
).to_csv("section_low_grade.csv")

In [47]:
# Read extreme peaks
extreme_peaks = pd.read_csv("extreme peaks.csv", header=None).iloc[:,
                                                                   0].to_list()
# Get top 1000 peaks values
mzs_top_k = mzs[((mzs >= 600) & (mzs <= 900))][top_k_peaks]
# Get indexes of extreme peaks in top 1000 peaks values
indexes_extreme_peaks = np.isin(np.round(mzs_top_k + 0.00001, 4), extreme_peaks)
# Get extreme peaks in each section replica and save to csv
pd.DataFrame(
    {k: v[indexes_extreme_peaks] for k, v in mean_top_k.items()},
    index=mzs_top_k[indexes_extreme_peaks]
).sort_index().to_csv("extreme peaks per biopsy average.csv")

***Average, std, count of spectras***
1. Calculate average of all spectras.
2. Find top 1000 peaks.
4. For replica\section:
  4.1. For high\low grade:
    4.1.1. Get the intensities for top 1000 for all spectras.
    4.2.1. Get average, std and number of spectras for what we got in 4.1.1

***Average, std, count of biopsies averages***
1. Calculate average for each biopsy.
2. Calculate average of all averages (global average).
3. Find top 1000 mzs in the global average.
4. For replica\section:
  4.1. For high\low grade:
  4.1.1. Get intensities for top 1000 mzs in each biopsy average.
  4.2.1. Get average, std and number of biopsies for what we got in 4.1.1.